In [1]:
from ib.ext.Contract import Contract
from ib.ext.ScannerSubscription import ScannerSubscription
from ib.ext.ContractDetails import ContractDetails
from ib.ext.Order import Order
from ib.opt import Connection, message

from time import sleep 
from datetime import datetime
import pandas as pd
import numpy as np
# import backtrader as bt
# import pytz, tzlocal
from bs4 import BeautifulSoup
import re
from ibapi.ticktype import TickTypeEnum
import talib

In [2]:
def error_handler(msg):
    """Handles the capturing of error messages"""
    print ("Server Error: {}".format(msg))

def reply_handler(msg):
    """Handles of server replies"""
    print ("Server Response: {}, {}".format(msg.typeName, msg))

def next_id(msg):
    global order_id
    order_id = int(re.findall('\d+', str(msg))[0])
    
    return order_id
    
def historical_data_handler(msg):  
    global historical_data
    
    if ('finished' in str(msg.date)) == False:   
        
        historical_data.loc[len(historical_data)] = ([msg.date,
                                                      msg.open, msg.high, msg.low, msg.close, msg.volume])
    else:  
        historical_data.set_index('Date',inplace=True)

        
def ReportsFinStatements(msg):
    global annual
    global quarter
   
    parser = BeautifulSoup(msg.data, 'lxml')

    mapp = dict()
    for item in parser.findAll('mapitem'):
        mapp[item['coaitem']] = item.text
        
    
    #ANNUAL
    index = [];columns = []
    for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
        index.append(datetime.strptime(item['fiscalyear'], '%Y').year)
    
    for i in parser.findAll('annualperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
        columns.append(mapp[i['coacode']])

    annual = pd.DataFrame(index = index, columns = columns)
    annual.index.name = 'Annual'
    for item in parser.findAll('annualperiods')[0].findAll('fiscalperiod'):
        for i in item.findAll('lineitem'):
            try:
                annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = float(i.text)
            except:
                annual.loc[datetime.strptime(item['fiscalyear'], '%Y').year,mapp[i['coacode']]] = np.nan
        
    # QUARTER
    index_q = [];columns_q = []
    for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
        index_q.append(datetime.strptime(item['enddate'], '%Y-%m-%d').date())
    
    for i in parser.findAll('interimperiods')[0].findAll('fiscalperiod')[0].findAll('lineitem'):
        columns_q.append(mapp[i['coacode']])

    quarter = pd.DataFrame(index = index_q, columns = columns_q)
    quarter.index.name = 'Quarter'
    for item in parser.findAll('interimperiods')[0].findAll('fiscalperiod'):
        for i in item.findAll('lineitem'):
            try:
                quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = float(i.text)
            except:
                quarter.loc[datetime.strptime(item['enddate'], '%Y-%m-%d').date(),mapp[i['coacode']]] = np.nan


def updatePortfolio(msg):
    global positions
    positions.append(msg)

def mktdata(msg):
    global news
    news.append(msg)
    
def Scanner(msg):
    global symbol
    global sectype
    global exchange
    global currency
        
    symbol.append(msg.contractDetails.m_summary.m_symbol)
    sectype.append(msg.contractDetails.m_summary.m_secType)
    exchange.append(msg.contractDetails.m_summary.m_exchange)
    currency.append(msg.contractDetails.m_summary.m_currency)

def Analyst_Estimates(msg):
    global resc
    global resc_ann
    global resc_q
    
    parser = BeautifulSoup(msg.data, 'lxml')
    resc.iloc[:,0] = parser.find('name').text    
    resc.iloc[:,1] = parser.find('exchange').text
    resc.iloc[:,2] = parser.findAll('secid')[0].text
    resc.iloc[:,3] = parser.findAll('sector')[0].text

    resc.iloc[:,4] = parser.findAll('marketdataitem')[0].text
    resc.iloc[:,5] = parser.findAll('marketdataitem')[1].text
    resc.iloc[:,6] = parser.findAll('marketdataitem')[2].text
    resc.iloc[:,7] = parser.findAll('marketdataitem')[3].text
    resc.iloc[:,8] = parser.findAll('marketdataitem')[4].text
    resc.iloc[:,9] = parser.findAll('fyactual')[0].text
    
    annual = [];columns = [];quarter = []
    for item in parser.findAll('fyactual'):
        columns.append(item['type'])
        for per in item.findAll('fyperiod'):
            if per['periodtype'] == 'A':
                annual.append(per['fyear'])
            if per['periodtype'] == 'Q':
                quarter.append('{}-{}'.format(per['endcalyear'],per['endmonth']))
    
    index = list(set(annual))
    index_q = list(set(quarter))
    resc_ann = pd.DataFrame(index = index, columns = columns).sort_index(axis = 0,ascending = False)
    resc_q = pd.DataFrame(index = index_q, columns = columns).sort_index(axis = 0,ascending = False)
    
    for item in parser.findAll('fyactual'):
        for per in item.findAll('fyperiod'):
            
            if per['periodtype'] == 'Q':
                try:
                    resc_q.loc['{}-{}'.format(per['endcalyear'],per['endmonth']),item['type']] = float(per.find('actvalue').text)
                except:
                    resc_q.loc['{}-{}'.format(per['endcalyear'],per['endmonth']),item['type']] = np.nan  
                    
            if per['periodtype'] == 'A':
                try:
                    resc_ann.loc[per['fyear'],item['type']] = float(per.find('actvalue').text)
                except:
                    resc_ann.loc[per['fyear'],item['type']] = np.nan

                    
def ReportsFinSummary(msg):
    global fin
    
    parser = BeautifulSoup(msg.data, 'lxml')
    date_div = [];data_div = []
    
    for i in parser.findAll('dividendpershare'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            
            date_div.append(i['asofdate'])
            try:
                data_div.append(float(i.text))
            except:
                data_div.append(np.nan)
                
    data_rev = []
    for i in parser.findAll('totalrevenue'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            try:
                data_rev.append(float(i.text))
            except:
                data_rev.append(np.nan)
                
    data_eps = []
    for i in parser.findAll('eps'):
        if i['period'] == '12M' and i['reporttype'] == 'TTM':
            try:
                data_eps.append(float(i.text))
            except:
                data_eps.append(np.nan)

    fin = pd.DataFrame(index = date_div,
                       data = {'Dividend Per Share(TTM)':data_div,
                               'Total Revenue(TTM)':data_rev, 
                               'EPS(TTM)':data_eps}).sort_index(axis = 0,ascending = False)

def ReportSnapshot(msg):
    global business_summary
    global brief
    global snap
    global address
    global ratio
    global estimate
    
    parser = BeautifulSoup(msg.data, 'lxml')
    
    business_summary = parser.findAll('text')[0].text
    brief = parser.findAll('text')[1].text
    address = '{},{},{},{}'.format(parser.find('contactinfo').find('streetaddress').text,
                                   parser.find('contactinfo').find('city').text,
                                   parser.find('contactinfo').find('state-region').text,
                                   parser.find('contactinfo').find('country').text)
    
    snap = pd.DataFrame(index = [parser.findAll('coid')[1].text],
                        data = {'Company Type': parser.find('cotype').text,
                                'Desc':parser.findAll('issue')[0]['desc'],
                                'Exchange': parser.findAll('exchange')[0].text,
                                'Industry':parser.findAll('industry')[0].text,
                                'Index':parser.find('indexconstituet').text})
    dicted = {};dicted_est = {}
    for i in range(len(parser.find('ratios').findAll('ratio'))):
        try:
            dicted[parser.find('ratios').findAll('ratio')[i]['fieldname']] = float(parser.find('ratios').findAll('ratio')[i].text)
        except:
            dicted[parser.find('ratios').findAll('ratio')[i]['fieldname']] = parser.find('ratios').findAll('ratio')[i].text
            
    ratio = pd.DataFrame(index = [parser.findAll('coid')[1].text], data = dicted)
    
    for i in range(len(parser.find('forecastdata').findAll('ratio'))):
        try:
            dicted_est[parser.find('forecastdata').findAll('ratio')[i]['fieldname']] = float(parser.find('forecastdata').findAll('ratio')[i].text)
        except:
            dicted_est[parser.find('forecastdata').findAll('ratio')[i]['fieldname']] = parser.find('forecastdata').findAll('ratio')[i].text
            
    estimate = pd.DataFrame(index = [parser.findAll('coid')[1].text], data = dicted_est)
    
def create_contract(symbol, sec_type, exch, curr):

    contract = Contract()
    contract.m_symbol = symbol
    contract.m_secType = sec_type
    contract.m_exchange = exch
    #contract.m_primaryExch = prim_exch
    contract.m_currency = curr
    
    return contract

def create_order(order_type, quantity, action):

    order = Order()
    order.m_orderType = order_type
    order.m_totalQuantity = quantity
    order.m_action = action
    
    return order

def create_scanner(numberOfRows, instrument, locationCode, ScanCode, 
                   AbovePrice, marketCapAbove, AboveVolume):

    scanSub = ScannerSubscription()
    scanSub.numberOfRows(numberOfRows)
    scanSub.m_instrument = instrument
    scanSub.m_locationCode = locationCode
    scanSub.m_scanCode = ScanCode
    scanSub.m_abovePrice = AbovePrice
    scanSub.m_marketCapAbove = marketCapAbove
    scanSub.m_aboveVolume = AboveVolume
    
    return scanSub



In [3]:
#price = pd.DataFrame(columns = ['Ticker Id','Field','Price'])
#size = pd.DataFrame()
#scan_param = None
#tws_conn.register(ScannerParameters, message.scannerParameters)
#tws_conn.register(mktdata, message.tickPrice)
#tws_conn.reqMarketDataType(4) # switch to delayed frozen data if live is not available
#tws_conn.reqScannerParameters()

# Connecting to TWS

In [4]:
tws_conn = Connection.create(host='127.0.0.1',port=7497, clientId=0)
tws_conn.connect()

Server Version: 76
TWS Time at connection:20191014 05:10:45 EST


True

In [5]:
tws_conn.register(error_handler, 'Error')
tws_conn.register(reply_handler)
# tws_conn.unregister(error_handler, 'Error')
# tws_conn.unregister(reply_handler)

False

In [6]:
#Briefing Trader
contract_brief = create_contract(symbol = 'BRFG:BRFG_ALL',
                sec_type = 'NEWS',
                exch = 'BRFG', curr='')
#Benzinga Pro
contract_benzinga = create_contract(symbol = 'DJNL:DJNL_ALL',
                sec_type = 'NEWS',
                exch = 'DJNL', curr='')
#Midnight Trader
contract_midnight = create_contract(symbol = 'BRFUPDN:BRF_ALL',
                sec_type = 'NEWS',
                exch = 'BRFUPDN', curr='')

In [7]:
news=[]

In [8]:
tws_conn.register(mktdata, message.tickString)
tws_conn.reqMktData(0, contract_brief, "mdoff,292", False)
tws_conn.reqMktData(1, contract_benzinga, "mdoff,292", False)
tws_conn.reqMktData(2, contract_midnight, "mdoff,292", False)

sleep(5)

In [9]:
news

In [14]:
print (news[0].value)
print (news[1].value)
print (news[2].value)
print (news[3].value)
print (news[4].value)
print (news[5].value)

BRFG$0bbfa23b 20191011-19:58:00 BRFG Trade sensitive sectors leading the market higher as U.S.-China trade tensions ease
BRFG$0bbfffcd 20191011-20:14:46 BRFG TechStocks
BRFG$0bc0065f 20191011-20:27:00 BRFG Weekly Wrap
BRFUPDN$0bbf1fe3 20191011-13:18:50 BRFUPDN Morgan Stanley initiated Ascendis Pharma (ASND) coverage with Overweight and target $128
DJNL$025ab6b3 20191011-10:49:56 DJNL North American Morning Briefing
DJNL$08aff51c 20191011-20:43:26 DJNL Tomorrow's News Today


In [15]:
tws_conn.unregister(mktdata, message.tickString)

False

# Step 1 : Filtertation

In [16]:
symbol = [];sectype = [];exchange = []; currency = []

In [17]:
ScanCode = 'LOW_VS_52W_HL'
scanSub = create_scanner(numberOfRows = 10, instrument = 'STK', 
                         locationCode = 'STK.US.MAJOR', ScanCode = ScanCode, 
                         AbovePrice = '100',marketCapAbove = '100000',
                         AboveVolume = '1000')


In [18]:
tws_conn.register(Scanner, message.scannerData)
tws_conn.reqScannerSubscription(1,scanSub)
sleep(5)
tws_conn.unregister(Scanner,message.scannerData)

Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>


True

Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>
Server Error: <error id=1, errorCode=165, errorMsg=Historical Market Dat

In [9]:
scanner = pd.DataFrame()
scanner['Symbol'] = symbol
scanner['SecType'] = sectype
scanner['Exchange'] = exchange
scanner['Currency'] = currency
scanner.set_index('Symbol', inplace=True)

In [10]:
scanner

,SecType,Exchange,Currency
Symbol,,,


Server Error: <error id=0, errorCode=165, errorMsg=Historical Market Data Service query message:no items retrieved>


In [11]:
contracts = []
for stock in scanner.index:
    
    contract = create_contract(symbol = stock,
                   sec_type = scanner.loc[stock,'SecType'],
                   exch = scanner.loc[stock,'Exchange'],
                   curr = scanner.loc[stock,'Currency'])
    contracts.append(contract)

In [12]:
contracts

Server Error: <error id=None, errorCode=None, errorMsg=>


## Technical Data

In [13]:
technical = []
for contract in contracts:
    historical_data = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    tws_conn.register(historical_data_handler, message.historicalData)
    tws_conn.reqHistoricalData(0, contract,'','1 M','1 hour','TRADES',1,1)
    sleep(5)

    historical_data['RSI'] = talib.RSI(historical_data['Close'], timeperiod=14)
    historical_data['Return'] = (historical_data['Close'] - historical_data['Close'].shift(1))/historical_data['Close']
    historical_data['macd'], historical_data['macdsignal'], historical_data['macdhist'] = talib.MACDEXT(historical_data['Close'], fastperiod=12, fastmatype=1, slowperiod=26, slowmatype=1, signalperiod=9, signalmatype=1)
    
    technical.append(historical_data)
    tws_conn.unregister(historical_data_handler, message.historicalData)

In [14]:
len(technical)

10

In [16]:
technical[0].tail(10)

,Open,High,Low,Close,Volume,RSI,Return,macd,macdsignal,macdhist
Date,,,,,,,,,,
20191010 13:00:00,152.54,153.13,152.29,152.97,74,41.178443,0.003792,-0.909491,-0.821520,-0.087971
20191010 14:00:00,153.09,153.58,152.68,152.68,53,39.833324,-0.001899,-0.909572,-0.839130,-0.070442
20191010 15:00:00,152.72,152.84,152.18,152.57,392,39.308807,-0.000721,-0.908045,-0.852913,-0.055132
20191011 09:30:00,153.06,154.29,152.21,152.71,198,40.384754,0.000917,-0.885333,-0.859397,-0.025935
20191011 10:00:00,152.69,155.00,152.68,153.95,203,49.007572,0.008055,-0.758531,-0.839224,0.080693
20191011 11:00:00,153.83,154.65,153.13,154.19,308,50.499927,0.001557,-0.631396,-0.797658,0.166263
20191011 12:00:00,153.90,154.74,153.68,154.74,93,53.834355,0.003554,-0.480718,-0.734270,0.253552
20191011 13:00:00,154.75,155.16,153.95,154.60,294,52.858290,-0.000906,-0.368356,-0.661087,0.292732
20191011 14:00:00,154.52,154.81,154.02,154.37,311,51.215416,-0.001490,-0.294473,-0.587764,0.293292


# Step 3: Apply Strategy

# Step 4: Position Account

In [6]:
positions = []

In [8]:
tws_conn.register(updatePortfolio, message.updatePortfolio)

True

In [9]:
tws_conn.reqAccountUpdates(True,'DU1440139')

In [28]:
contracts_pos = []
for i in range(int(len(positions)/5)):
    
    print (positions[i].contract.m_symbol, positions[i].position, 
           np.round(positions[i].marketPrice, 2), positions[i].averageCost, 
           positions[i].unrealizedPNL, positions[i].realizedPNL)
    
    contracts_pos.append(positions[i].contract.m_symbol)

AAPL 10 237.25 220.76 164.9 0.0
NVDA 20 186.55 184.7 37.0 0.0


In [17]:
tws_conn.unregister(updatePortfolio, message.updatePortfolio)

True

# Step 5: Placing Orders

In [ ]:
order_id = None

In [26]:
for contract in contracts:
    if contract.m_symbol in contracts_pos:
        pass
    else:
        order = create_order(order_type = 'MKT',
                     quantity = 10, 
                     action = 'BUY')
        
        tws_conn.register(next_id, message.nextValidId)
        tws_conn.reqIds(0)
        tws_conn.placeOrder(order_id, contract, order)

In [27]:
print ('Creating Order: ')
print (' Ticker: ',contract.m_symbol)
print (' SecType: ',contract.m_secType)
print (' Exchange: ',contract.m_exchange)
print (' primExchange: ',contract.m_primaryExch)
print (' Order Quantity: ',order.m_totalQuantity,'\n','Order Type: ', order.m_orderType,'\n','Order Action: ',order.m_action)

Creating Order: 
 Ticker:  AAPL
 SecType:  STK
 Exchange:  SMART
 primExchange:  NASDAQ
 Order Quantity:  10 
 Order Type:  MKT 
 Order Action:  BUY


In [15]:
tws_conn.placeOrder(order_id, contract, order)

Server Error: <error id=62, errorCode=399, errorMsg=Order Message:
BUY 10 AAPL NASDAQ.NMS
Server Error: <error id=62, errorCode=202, errorMsg=Order Canceled - reason:>


# Disconnect

In [20]:
tws_conn.disconnect()

False